In [29]:
import os
import json
import tensorflow as tf
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
# from __future__ import absolute_import
# from __future__ import division
# from __future__ import print_function

import argparse
import numpy as np
from sklearn import linear_model
import tensorflow.compat.v1 as tf
from tensorflow.compat.v1 import keras
from tensorflow.compat.v1.keras import applications
from tensorflow.compat.v1.keras import layers
from tensorflow.compat.v1.keras import models

In [30]:
images_dir =  "./images"
json_path = "./classes_name.json"
label_dict = json.load(open(json_path, "r"))
csv_path = "./new_train.csv"
csv_data = pd.read_csv(csv_path)

total_num = csv_data.shape[0]

img_paths = [os.path.join(images_dir, i)for i in csv_data["filename"].values]

img_label = [label_dict[i][0] for i in csv_data["label"].values]

labels = set(csv_data["label"].values) # 共有多少类

In [31]:
image_raw_data = tf.compat.v1.gfile.FastGFile('./images/n0153282900000005.jpg','rb').read()
image = tf.image.decode_jpeg(image_raw_data) #图片解码
with tf.compat.v1.Session() as sess:
    image_arr = image.eval()

# image_arr = image.numpy()

image_expand = np.expand_dims(image_arr,axis=0)
temp = encode_image(image_expand, encoder_model, input_shape, preprocess_function)
enc_x_train = temp

2022-04-26 22:49:46.852614: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-26 22:49:46.852707: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-04-26 22:49:47.742710: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-26 22:49:47.883813: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-26 22:49:47.883841: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_fac

In [ ]:
i=1
for i in range(3):
    image_raw_data = tf.compat.v1.gfile.FastGFile(img_paths[i],'rb').read()
    image = tf.image.decode_jpeg(image_raw_data) #图片解码
    with tf.compat.v1.Session() as sess:
        image_arr = image.eval()
        
    image_expand = np.expand_dims(image_arr,axis=0)
    temp = encode_image(image_expand, encoder_model, input_shape, preprocess_function)
    enc_x_train = np.concatenate((enc_x_train,temp),axis=0)

2022-04-26 22:49:58.370748: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-26 22:49:58.370768: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-04-26 22:49:59.458522: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-26 22:49:59.646982: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-26 22:49:59.647004: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_fac

In [2]:
# Encodes samples
# The preprocessing function used on the pre-training dataset is also
# applied while encoding the inputs.
preprocess_function = applications.inception_v3.preprocess_input
input_shape = (299, 299)

def encoder_model(architecture='inception_v3', pre_trained_dataset='imagenet',
                      downsample_factor=8):
    """Returns encoder model.
    Defines the encoder model to learn the representations for image dataset.
    In this example, we are considering the InceptionV3 model trained on
    ImageNet dataset, followed by simple average pooling-based downsampling.
    Args:
      architecture: Base architecture of encoder model (e.g. 'inception_v3')
      pre_trained_dataset: The dataset used to pre-train the encoder model
      downsample_factor: Downsample factor for the outputs
    Raises:
      NameError: Returns name errors if architecture is not 'inception_v3'
    """
    tf_input = layers.Input(shape=(input_shape[0], input_shape[1], 3))
    if architecture == 'inception_v3':
        model = applications.inception_v3.InceptionV3(
            input_tensor=tf_input, weights=pre_trained_dataset, include_top=False)
        output_pooled = \
            layers.AveragePooling2D((downsample_factor, downsample_factor),
                                    strides=(downsample_factor,
                                             downsample_factor))(model.output)
    else:
        raise NameError('Invalid architecture')
    return models.Model(model.input, output_pooled)

In [24]:
import io
import os

import zipfile

import numpy as np
import pandas as pd

from six.moves import urllib
from sklearn import preprocessing
# import tensorflow.compat.v1 as tf
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.compat.v1.keras import backend
from tensorflow.compat.v1.keras import datasets
def encode_image(features, encoder_model, input_shape, preprocess_function,
                 batch_size=16):
    """Encodes images using pre-trained encoder model.
    We apply encoding step with a pre-trained model and save the encoded
    representation to input them
    directly to DVRL, for computational efficiency.
    Args:
      features: Input image
      encoder_model: Model for encoding images (e.g., InceptionV3)
      input_shape: Input shape of the encoder model
      preprocess_function: Preprocessing function from features to input_shape of
                           encoder_model
      batch_size: Number of mini-batches for encoding images
    Returns:
      encoded_features: Encoded images
    """

    # Number of samples
    n_features = len(features)

    # Placeholder for batch of images
    batch_of_images_placeholder = tf.compat.v1.placeholder('uint8',
                                                 (None, features.shape[1],
                                                  features.shape[2],
                                                  features.shape[3]))
    
#     batch_of_images_placeholder = (None, features.shape[1],features.shape[2],features.shape[3])
    
    tf_resize_op = tf.compat.v1.image.resize_images(batch_of_images_placeholder,
                                          input_shape, method=0)

    # Data extraction function
    def data_generator(sess, data):
        """Generates preprocessed data.
        Args:
          sess: Session
          data: Image data
        Returns:
          generator: Generator function
        """

        def generator():
            """Subfunction of data generator in the session.
            Yields:
              batch_of_image_preprocessed: preprocessed data
            """
            start = 0
            end = start + batch_size
            n = data.shape[0]
            while True:
                batch_of_images_resized = sess.run(tf_resize_op,
                                                   {batch_of_images_placeholder:
                                                        data[start:end]})
                batch_of_images__preprocessed = \
                    preprocess_function(batch_of_images_resized)
                start = start + batch_size
                end = end + batch_size
                if start >= n:
                    start = 0
                    end = batch_size
                yield batch_of_images__preprocessed

        return generator

    with tf.compat.v1.Session() as sess:

        backend.set_session(sess)
        model = encoder_model()
        data_gen = data_generator(sess, features)
        ftrs_training = model.predict_generator(data_gen(),
                                                n_features // batch_size + 1,
                                                verbose=1)

    encoded_features = \
        np.array([ftrs_training[i].flatten() for i in range(n_features)])

    return encoded_features
